In [40]:
import os
import _pickle as pickle
import pandas as pd                                    
import numpy as np                    
from collections import defaultdict
import folium
from folium import plugins
from pandas import DataFrame, Series                
#matplotlib inline
from elasticsearch import Elasticsearch, helpers



In [41]:
import json
issues_pulled = [json.loads(line) for line in open('SPM587SP18issues.json')]#Loading the json file of the issues created in the git repo

In [42]:
issues_df = DataFrame(issues_pulled)#Putting the issues into a panda dataframe

In [43]:
issues_df #Printing issues_df


,Author,State,closed_at,created_at,issue_number,labels
0,HSP18SCM50W,closed,2018-04-22,2018-04-20,475,"[Category:Inquiry, DetectionPhase:Field, Origi..."
1,HSP18SCM50W,closed,2018-04-22,2018-04-19,474,"[Address:59 W Grand Ave Chicago IL 60654, Cate..."
2,YSP18SCM40K,closed,2018-04-14,2018-04-14,472,"[Category:Inquiry, DetectionPhase:Field, Origi..."
3,RSP18SCM19N,closed,2018-04-14,2018-04-14,470,"[Category:Enhancement, DetectionPhase:Testing,..."
4,CSP18SCM32L,closed,2018-04-13,2018-04-13,466,"[Category:Inquiry, DetectionPhase:Field, Origi..."
5,RSP18SCM19N,closed,2018-04-13,2018-04-13,464,"[Address: 2400 N Lincoln Ave Chicago IL 60614,..."
6,MSP18SCM65B,closed,2018-04-13,2018-04-13,461,"[Category:Bug, DetectionPhase:Testing, Origina..."
7,JSP18SCM63J,closed,2018-04-13,2018-04-13,459,[]
8,YSP18SCM35Z,closed,2018-04-13,2018-04-13,454,"[Category:Enhancement, DetectionPhase:Testing,..."
9,YSP18SCM40K,closed,2018-04-13,2018-04-13,452,"[Address:225 S Canal St Chicago IL 60606, Cate..."


In [44]:
wrangled_issues_df = issues_df[['Author','State','closed_at','created_at','issue_number','labels']]# as per the code given in the tutorial,filtering and arranging the Dataframe 
wrangled_issues_df.loc[0:len(wrangled_issues_df), 'OriginationPhase']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'DetectionPhase']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'Category']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'Priority']= np.NaN
wrangled_issues_df.loc[0:len(wrangled_issues_df),'Status']= np.NaN


In [45]:
newList = list() #creating a new list
for i in range(0, len(wrangled_issues_df)):#Since in the json file, the labels are not in form of key value pair, but an array of string, they cannot be accessed
#thus the label part of dataframe is split into a new dictioanry of key value pair and updated into the new list.
    tempDictionary = dict()
    if wrangled_issues_df.iloc[i]['labels']:
        for label in wrangled_issues_df.iloc[i]['labels']:
            label_name= (label.split(':'))[0]
            label_value= (label.split(':'))[1]
            tempDictionary.update({label_name : label_value})
    tempDictionary.update({'issue_number' : int(wrangled_issues_df.iloc[i]['issue_number'])})#Since the panda dataframe uses numpy iteger, casting it into
    # primitive integer, as elastic search only accepts primitve data types.
    newList.append(tempDictionary) 

In [46]:
#newList

In [47]:
actions = list() #updating elastic search database as per given 
es = Elasticsearch()
for data in newList:
    action = {
        '_index':'issues_database',
        '_type':'gitRepo',
        '_id':data['issue_number'],
        '_source':data
    }
    actions.append(action)
helpers.bulk(es,actions)
    

(255, [])

In [48]:
first_query = { #first query where all the issues are pulled from the database
    'size' : 500,
    'query' : {
        'match_all' : {}
    }
}
queried_output_first = es.search(index = 'issues', body=first_query,scroll='1h') #issues are stored in json format

In [80]:
queried_output_first['hits']['hits'][0]

{'_id': '470',
 '_index': 'issues',
 '_score': 1.0,
 '_source': {'Category': 'Enhancement',
  'DetectionPhase': 'Testing',
  'OriginationPhase': 'Design',
  'Priority': 'Major',
  'Status': 'Completed',
  'issue_number': 470},
 '_type': 'shreyas'}

In [50]:
sid = queried_output_first['_scroll_id'] #As per the tutorial code, setting the scroll size.
scroll_size = queried_output_first['hits']['total']

In [51]:
count = 0
first_query_coord = []
while(scroll_size > count):
    for doc in queried_output_first['hits']['hits']:  #As per the code in the tutorial, accessing the values in the key(data['hits']['hits'])
# This the key value format generated by the elastic search when pulled from the database. The value of this key contains all the various labels.
        location_ll = []
        results = doc['_source']
        count = count +1
        if 'Latitude' in results.keys():
            if 'Longitude' in results.keys():
                if(results['Latitude'] != None and results['Longitude'] != None):
                    location_ll.append(float(results['Latitude']))
                    location_ll.append(float(results['Longitude']))
                    first_query_coord.append(location_ll)
    
                    

In [52]:
print(len(first_query_coord))

137


In [53]:
first_query_heat_map = folium.Map([41.891551, -87.607375],zoom_start = 16)
first_query_heat_map.add_child(plugins.HeatMap(first_query_coord,radius=15))

In [54]:
second_query = { #second query to match the labels given
    'size' : 500,
    'query' : {
        'bool':{
            'must' : [{'match':{'DetectionPhase':'Field'}},
                      {'match':{'Priority':'Critical'}}]
        }
    }
}
queried_output_second = es.search(index = 'issues', body=second_query,scroll='1h')#This is a dictionary variable. Pulls data and stores as key value pair in the dictionary.

In [55]:
sid = queried_output_second['_scroll_id']
scroll_size = queried_output_second['hits']['total']

In [56]:
count = 0 #As per the tutorial, accessing the Latitude and Longitude values from key, and storing them into an array. The if else condition checks
#if the issues have those values or not, if not then that issue is skipped.
second_query_coord = []
while(scroll_size > count):
    for doc in queried_output_second['hits']['hits']:
        location_ll = []
        results = doc['_source']
        count = count +1
        if 'Latitude' in results.keys():
            if 'Longitude' in results.keys():
                if(results['Latitude'] != None and results['Longitude'] != None):
                    location_ll.append(float(results['Latitude']))
                    location_ll.append(float(results['Longitude']))
                    second_query_coord.append(location_ll)
    
                    

In [57]:
print(len(second_query_coord))

3


In [58]:
second_query_heat_map = folium.Map([41.891551, -87.607375],zoom_start = 16) #Syntax of folium map. Setting the default location tp view.
second_query_heat_map.add_child(plugins.HeatMap(second_query_coord,radius=15)) #plot the coordinates onto the map.

In [59]:
third_query = { #Third query to match the given labels 
    'size' : 500,
    'query' : {
        'bool':{
            'must' : [{'match':{'DetectionPhase':'Field'}},
                      {'match':{'Status':'Completed'}}]
        }
    }
}
queried_output_third = es.search(index = 'issues', body=third_query,scroll='1h')#This is a dictionary variable. Pulls data and stores as key value pair in the dictionary.

In [60]:
sid = queried_output_third['_scroll_id']
scroll_size = queried_output_third['hits']['total']

In [61]:
count = 0
third_query_coord = []
while(scroll_size > count):
    for doc in queried_output_third['hits']['hits']:
        location_ll = []
        results = doc['_source']
        count = count +1
        if 'Latitude' in results.keys():
            if 'Longitude' in results.keys():
                if(results['Latitude'] != None and results['Longitude'] != None):
                    location_ll.append(float(results['Latitude']))
                    location_ll.append(float(results['Longitude']))
                    third_query_coord.append(location_ll)
    
                    

In [62]:
print(len(third_query_coord))

4


In [63]:
third_query_heat_map = folium.Map([41.891551, -87.607375],zoom_start = 16)
third_query_heat_map.add_child(plugins.HeatMap(third_query_coord,radius=15))

In [64]:
fourth_query = {#Fourth query to match the given labels 
    'size' : 500,
    'query' : {
        'bool':{
            'must' : [{'match':{'DetectionPhase':'Field'}},
                      {'match':{'Priority':'Critical'}}, 
                      {'match':{'Status':'Approved'}}]
        }
    }
}
queried_output_fourth = es.search(index = 'issues', body=fourth_query,scroll='1h')#This is a dictionary variable. Pulls data and stores as key value pair in the dictionary.

In [65]:
sid = queried_output_fourth['_scroll_id']
scroll_size = queried_output_fourth['hits']['total']

In [66]:
count = 0
fourth_query_coord = []
while(scroll_size > count):
    for doc in queried_output_fourth['hits']['hits']:
        location_ll = []
        results = doc['_source']
        count = count +1
        if 'Latitude' in results.keys():
            if 'Longitude' in results.keys():
                if(results['Latitude'] != None and results['Longitude'] != None):
                    location_ll.append(float(results['Latitude']))
                    location_ll.append(float(results['Longitude']))
                    fourth_query_coord.append(location_ll)
    
                    

In [67]:
print(len(fourth_query_coord))

2


In [68]:
fourth_query_heat_map = folium.Map([41.891551, -87.607375],zoom_start = 16)
fourth_query_heat_map.add_child(plugins.HeatMap(fourth_query_coord,radius=15))

In [69]:
fifth_query = {#Fifth query to match the given labels 
    'size' : 500,
    'query' : {
        'bool':{
            'must' : [{'match':{'DetectionPhase':'Field'}},
                      {'match':{'Priority':'Critical OR High'}},
                      {'match':{'Status':'Approved OR inProgress'}}]
    }
}
}
queried_output_fifth = es.search(index = 'issues', body=fifth_query,scroll='1h')#This is a dictionary variable. Pulls data and stores as key value pair in the dictionary.

In [70]:
sid = queried_output_fifth['_scroll_id']
scroll_size = queried_output_fifth['hits']['total']

In [71]:
count = 0
fifth_query_coord = []
while(scroll_size > count):
    for doc in queried_output_fifth['hits']['hits']:
        location_ll = []
        results = doc['_source']
        count = count +1
        if 'Latitude' in results.keys():
            if 'Longitude' in results.keys():
                if(results['Latitude'] != None and results['Longitude'] != None):
                    location_ll.append(float(results['Latitude']))
                    location_ll.append(float(results['Longitude']))
                    fifth_query_coord.append(location_ll)
    
                    

In [72]:
print(len(fifth_query_coord))

4


In [73]:
fifth_query_heat_map = folium.Map([41.891551, -87.607375],zoom_start = 16)
fifth_query_heat_map.add_child(plugins.HeatMap(fifth_query_coord,radius=15))

In [74]:
sixth_query = { #Sixth query to match the given labels 
    'size' : 500,
    'aggs' : {
        'data' : {
            'terms' : {
                'field' : 'Latitude.keyword',
                'field' : 'Longitude.keyword',
                'min_doc_count' : 5,
                'size' : 500
            },
            'aggs' : {
                'top_selection' : {
                    'top_hits' : {
                        'size' : 10
                    }
                }
            }
        }
    }
}
queried_output_sixth = es.search(index = 'issues', body=sixth_query,scroll='1h') #This is a dictionary variable. Pulls data and stores as key value pair in the dictionary.


In [75]:
sid = queried_output_sixth['_scroll_id']
scroll_size = queried_output_sixth['hits']['total']

In [76]:
count = 0
sixth_query_coord = []
for i in queried_output_sixth['aggregations']['data']['buckets']:
        location_ll = []
        results = i['top_selection']['hits']['hits'][0]['_source']
        if 'Latitude' in results.keys():
               if 'Longitude' in results.keys():
                if(results['Latitude'] != None and results['Longitude'] != None):
                    location_ll.append(float(results['Latitude']))
                    location_ll.append(float(results['Longitude']))
                    sixth_query_coord.append(location_ll)

In [77]:
print(len(sixth_query_coord))

14


In [78]:
sixth_query_heat_map = folium.Map([41.891551, -87.607375],zoom_start = 16)
sixth_query_heat_map.add_child(plugins.HeatMap(sixth_query_coord,radius=15))